# Python软件调试（debugging）

世界正处于数字化的浪潮之中，硬件正在变“软”、变的互联、变的智能。软件几乎已经无处不在，人类对软件的依赖日益增大。那么软件会犯错吗？人非圣贤孰能无过，软件是人编写的，自然也会犯错。

软件的错误或缺陷也称为臭虫（`Bug`）。名字的由来要回到上世纪，1947年9月9日，在测试Mark II计算机时突发故障。经过工作人员几个小时的检查，发现在面板F的第70号继电器中有一个死飞蛾。把飞蛾取出后，系统恢复正常。当时工作的著名的女科学家霍普（Grace Hopper）把飞蛾粘贴在工作手册中，并写了一行注释，“First actual case of bug being found”。自此之后，Bug就用来指代软件的错误或缺陷；而调试（debug，除虫）就用来泛指排除软件错误的过程。

![第一个计算机Bug](../images/debug_firstbug.jpg)

也许大家会觉得，发现臭虫一脚踩死不就完了，软件有Bug也没什么大不了的。软件Bug的历史也充满了人类鲜血和泪水。美国NASA的火星气候探测者号（Mars ClimateOrbiter），于1998年12月1日发射，在1999年9月23日到达预定轨道后失联。失联原因很简单：地面控制团队使用英制单位来发送导航指令，探测器的软件系统使用公制来读取指令，导致探测器进入低轨道后摩擦解体。损失高达3亿美元。因为软件错误缺陷导致的灾难举不胜举，可以阅读《致命Bug：软件缺陷的灾难与启示》一书了解更多。

如何编写安全代码，尽量避免软件错误，是一个程序员毕生需要修炼提高的。本节主要介绍Python软件调试的基本方法。

## 调试错误分类

当程序出现了问题，先别着急去调试。首先检查一下当前代码是否在版本控制中。如果是在开发状态中修改代码引起的bug，建议先添加一下更改，再进行调试；如果是发布版本引起的bug，建议使用Git创建一个调试分支，等解决问题后再合并到主分支。

在国内常常称调试为填坑，由于时间紧急工作匆忙，经常犯填完一个坑又挖了几个坑的问题。故调试完成后，建议好好测试一下，然后再发布。

在编写和运行Python程序的过程中，问题可以大致分为如下：
- 语法错误
- 运行错误
- 语义错误

### 语法错误（syntax error）

语法错误常见于初学者，常常是漏写一些符号，例如，在条件和循环语句后面忘记写冒号`:`，从其它地方拷贝过来代码，却忘记做适当修改。一般来说，语法错误容易修改，只要在出错代码行附近，仔细查看代码，定位错误，即可快速修复。

对于语法错误，PyCharm编辑器会自动检查并提示。建议每次提交更动前，把PyCharm给出的警告去全部处理完。另外还可以使用flake8工具对程序做静态分析。

下面列出一些新手常犯的错误

变量命名时违反命名规则：

In [9]:
21century = 2000

SyntaxError: invalid syntax (<ipython-input-9-60eeeb9c04ca>, line 1)

变量未定义即使用

In [10]:
score = 60
score += 1
grade += 1

NameError: name 'grade' is not defined

代码缩进不一致

In [1]:
def func():
    if score == 59：
        print('勉强通过')    
      return 

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 4)

忘记在`if`, `elif`, `else`, `for`, `while`, `class`,`def` 声明末尾添加`：`

In [11]:
if score == 59
    print('勉强通过')

SyntaxError: invalid syntax (<ipython-input-11-e84e0bc47456>, line 1)

在做条件判断时，使用`=`而不是`==`操作符

In [2]:
if score = 59:
    print('勉强通过')

SyntaxError: invalid syntax (<ipython-input-2-1b84b3ef0a80>, line 1)

字符串前后不闭合

In [3]:
print('Hello Python!")

SyntaxError: EOL while scanning string literal (<ipython-input-3-074e375d634d>, line 1)

函数或方法调用时括号没有成对闭合

In [4]:
print('PI={}'.format(3.131592)

SyntaxError: unexpected EOF while parsing (<ipython-input-4-790ff9a980c1>, line 1)

在编写代码时，混入中文输入字符

In [5]:
print('My name is '，"老王")

SyntaxError: invalid character in identifier (<ipython-input-5-33e71ed88c91>, line 1)

### 运行错误（`run-time error`）

Python程序的语法正确，但在运行时出现错误引发异常，导致程序终止，这类错误称为运行错误。Python解释器通常会抛出异常对象，同时用回溯（Traceback）来终止运行。对于经验丰富程序员来说，可以根据异常对象类型和回溯信息来定位错误，进而解决错误。

一些简单的运行错误通常是对所用对象用法不熟悉，错误使用而导致的。例如：
- 在调用除法运算时，除数为0；
- 访问列表元素时，索引出界；
- 访问字典元素时，使用错误键值；
- 打开文件时，文件不存在；
- 导入的模块并没有安装。

索引超出序列最大范围

In [6]:
colors = ['red', 'yellow', 'green']
print(colors[3])

IndexError: list index out of range

使用错误键值访问字典元素

In [7]:
color2value = {'red': 0, 'yellow': 2, 'green': 4}
color2value['RED']

KeyError: 'RED'

导入模块拼写错误或者未安装

In [8]:
import Numpy

ModuleNotFoundError: No module named 'Numpy'

然而很多时候，运行错误的原因隐晦不明，其根源未必就出错代码行附近，则需要使用更好的调试方法来解决。

### 语义错误（semantic error）

语义错误，也称为逻辑错误，是指程序运行状态或结果与预期不符合。对于此类错误，程序代码没有语法错误，在运行时可能并没有引起异常，甚至没有产生错误信息，但就是不能工作或结果错误。

语义错误通常隐藏的很深，并不容易发现。在发布的版本中，可能仍存在语义错误，一旦发生就容易引起致命错误。例如火星气候探测者号的错误就是语义错误，也是致命的错误。

大部分发生语义错误的情况是理解和沟通问题。一般来说，程序员都不是提出需求或设计程序的人。术业有专攻，程序员未必能够完全理解需求，如果再有沟通问题，很可能就会出现程序实现与需求或设计之间出现偏差，最终导致语义错误。

还有一些语义错误是由于程序员心手不一，出现的语义错误。程序员原意是想计算平均值，由于忘记了括号，导致语义错误

In [9]:
a, b = 1.0, 3.0
average = a + b / 2

使用字典方法`D.get()`时，如果指定键值不存在，缺省会返回'None'。而实际上程序员本意是返回指定值。

In [29]:
def getcolor(color):
    try:
        color2vlaue = {'red': 0, 'yellow': 2, 'green': 4}
        return color2value.get(color)
    except KeyError:
        raise NameError('Unknown color {}'.format(color))
        
getcolor('blue')        

很多时候，是由于语义错误最终导致运行错误的。

## 调试过程

在有记录的第一次计算机调试中，取出那只飞虫很任意，不过找到它却耗费了几个小时的时间。故而从一开始，软件调试就包括了定位错误与去除错误这两个基本步骤。一个完整的调试过程有如下步骤组成。

首先，**bug重现**。调试如同破案。一般出问题的软件运行在另一个地方的系统上，也就是说自己的调试系统并不是案发现场。所以，软件调试第一步就是在自己调试的系统上能够重现故障，就像案件重组一样。

第二，**定位根源**。综合利用各种调试工具与手段，寻找导致软件错误的根源。用户上报的通常是软件错误的外在表现，必须从外在出发，找到内在原因。是需求与实现不一致？还是程序员心手不一？还是对Python的错误使用？

第三，**bug修复**。根据寻找到的故障根源、紧迫程度，设计和实现修复bug方案。

第四，**修复验证**，对修复进行测试。首先验证上报Bug是否有效清除，然后验证此次修复没有导致新的Bug。

经过bug修复与修复验证的多次迭代，最终提交更改。

## 调试的特点

调试的难度大。调试要定位程序错误，就是从程序中逐步缩小搜索空间，最终锁定问题语句行或语句块。程序通常不会小，而且模块众多。这就要求程序员具有高超的分析能力，还得熟悉各个模块的架构和具体功能。此外，程序出现问题通常是程序员没想到的问题，通常是自己的软肋。

软件调试具有广泛的关联性，与计算机硬件、操作系统、编程语言等密切相关，这就要求程序员对各个环节都有所了解，经验丰富，方能融汇贯通，进而快速解决软件问题。

软件调试技术难度大，有很大的不确定性，故修复bug的时间也很难确定。软件项目工程又很紧张，常常导致软件问题简而化之或者不了了之。

## 调试工具

计算机最显著的一个特点就是“快”，现在日常用的计算机的计算速度也能达到一秒数亿次。而调试工具的一个特点就是“慢”，目的就是帮助人们来驾驭这台高速机器，利用调试工具，计算机可以暂停、慢速前进、直达目的地，任由程序员控制，最终发现软件 Bug。

对于 Python 软件调试，有如下工具：
- 日志输出
- 命令行调试工具pdb
- 图形界面调试工具pycharm

### 日志输出

很多时候，我们无法故障重现。那么日志就是程序提供的辅助调试手段。通过记录的日志信息，回放程序运行的过程，包括时间、地点与事件。通过日志信息，使得能够追踪软件问题发生的来龙去脉，找到案发的蛛丝马迹，最终找到故障根源。

尽管 pdb 或 pycharm 都提供有先进的调试工具，日志输出仍然不能不替代。所以写程序时，尽量能够使用`logging`模块来输出日志信息。

### pdb

pdb是Python内置的标准库之一，可以使用pdb来调试程序。pdb能够帮助我们准确的定位错误，发现程序中Bug。Pdb也有一些限制，例如无法进行远程调试或多线程调试。

### PyCharm调试器

PyCharm提供了先进的图形界面调试工具，使得用户可以轻松调试。同时PyCharm支持远程调试和多线程调试等功能。

## 更多

软件调试不仅仅是用来软件“除虫”的。使用软件调试，可以深入触碰和探索从计算机硬件、操作系统、编程语言到程序的方方面面。故而在《格囊汇编》一书作者张银奎更是建议，把软件调试作为编程学习的一把利剑，一种方法。

笔者深有同感。实际上，如果一个程序员不懂调试，很难谈得上对编程有深刻理解。这里推荐好朋友张银奎的两本书：
- 《格囊汇编》，张银奎
- 《软件调试》，张银奎